# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
import os, glob
from macrohet import dataio, tile, visualise
from tqdm.auto import tqdm

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
%%time
expt_ID = 'ND0001'

base_dir = f'/mnt/SYNO/macrohet_syno/{expt_ID}/'
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/ND0001__2023-10-27T14_12_54-Measurement 1/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 1min 32s, sys: 7.91 s, total: 1min 40s
Wall time: 1min 40s


id State                               URL Row Col  \
0          0301K1F1P1R1    Ok    r03c01f01p01-ch1sk1fk1fl1.tiff   3   1   
1          0301K1F1P1R2    Ok    r03c01f01p01-ch2sk1fk1fl1.tiff   3   1   
2          0301K1F1P1R3    Ok    r03c01f01p01-ch3sk1fk1fl1.tiff   3   1   
3          0301K1F1P2R1    Ok    r03c01f01p02-ch1sk1fk1fl1.tiff   3   1   
4          0301K1F1P2R2    Ok    r03c01f01p02-ch2sk1fk1fl1.tiff   3   1   
...                 ...   ...                               ...  ..  ..   
1020595  0612K300F9P2R2    Ok  r06c12f09p02-ch2sk300fk1fl1.tiff   6  12   
1020596  0612K300F9P2R3    Ok  r06c12f09p02-ch3sk300fk1fl1.tiff   6  12   
1020597  0612K300F9P3R1    Ok  r06c12f09p03-ch1sk300fk1fl1.tiff   6  12   
1020598  0612K300F9P3R2    Ok  r06c12f09p03-ch2sk300fk1fl1.tiff   6  12   
1020599  0612K300F9P3R3    Ok  r06c12f09p03-ch3sk300fk1fl1.tiff   6  12   

        FieldID PlaneID TimepointID ChannelID FlimID  ... PositionZ  \
0             1       1           0         1      1  ...    -2E-06   
1             1       1           0         2      1  ...    -2E-06   
2             1       1           0         3      1  ...    -2E-06   
3             1       2           0         1      1  ...         0   
4             1       2           0         2      1  ...         0   
...         ...     ...         ...       ...    ...  ...       ...   
1020595       9       2         299         2      1  ...         0   
1020596       9       2         299         3      1  ...         0   
1020597       9       3         299         1      1  ...     2E-06   
1020598       9       3         299         2      1  ...     2E-06   
1020599       9       3         299         3      1  ...     2E-06   

        AbsPositionZ MeasurementTimeOffset                        AbsTime  \
0        0.135358006                     0  2023-10-27T14:13:10.393+01:00   
1        0.135358006                     0  2023-10-27T14:13:10.393+01:00   
2        0.135358006                     0   2023-10-27T14:13:10.66+01:00   
3        0.135360003                     0   2023-10-27T14:13:10.94+01:00   
4        0.135360003                     0   2023-10-27T14:13:10.94+01:00   
...              ...                   ...                            ...   
1020595  0.135085002            266674.747  2023-10-30T15:32:33.063+00:00   
1020596  0.135085002            266674.747  2023-10-30T15:32:33.327+00:00   
1020597  0.135086998            266674.747   2023-10-30T15:32:33.61+00:00   
1020598  0.135086998            266674.747  2023-10-30T15:32:33.627+00:00   
1020599  0.135086998            266674.747   2023-10-30T15:32:33.89+00:00   

        MainExcitationWavelength MainEmissionWavelength  \
0                            561                    599   
1                            740                      0   
2                            640                    706   
3                            561                    599   
4                            740                      0   
...                          ...                    ...   
1020595                      740                      0   
1020596                      640                    706   
1020597                      561                    599   
1020598                      740                      0   
1020599                      640                    706   

        ObjectiveMagnification ObjectiveNA ExposureTime  \
0                           40         1.1          0.2   
1                           40         1.1          0.2   
2                           40         1.1          0.2   
3                           40         1.1          0.2   
4                           40         1.1          0.2   
...                        ...         ...          ...   
1020595                     40         1.1          0.2   
1020596                     40         1.1          0.2   
1020597                     40         1.1          0.2   
1020598                     40         1.1 

### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/ND0001__2023-10-27T14_12_54-Measurement 1/Assaylayout/*AssayLayout.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

In [4]:
acq_ID = row, column = (3, 4)

#### Define subset if non-square tiling or more than one contiguous region of images in imaging well. 

In [5]:
# subset_field_IDs = ['1','6','7','8','11','12','13','14','15']

#### Load images using Zarr (WIP)

In [6]:
import zarr

In [7]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')
images = zarr.open(image_dir, mode='r')

PathNotFoundError: nothing found at path ''

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [ ]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'acquisition/ND0001__2023-10-27T14_12_54-Measurement 1/Images')
dask_images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             # subset_field_IDs=['16', '17',  '20', '21'], 
                             # n_tile_rows = 2, n_tile_cols = 2,
                             set_plane='max_proj',
                             # set_channel=1,
                             # set_time = ,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()
dask_images

In [ ]:
for (row, column), info in assay_layout.iterrows():
    try:
        # image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
        image_dir = os.path.join(base_dir, 'acquisition/ND0001__2023-10-27T14_12_54-Measurement 1/Images')
        dask_images = tile.compile_mosaic(image_dir, 
                                     metadata, 
                                     row, column, 
                                     # subset_field_IDs=['16', '17',  '20', '21'], 
                                     # n_tile_rows = 2, n_tile_cols = 2,
                                     set_plane='max_proj',
                                     # set_channel=1,
                                     # set_time = ,
        #                             input_transforms = [input_transforms]
                                    )#.compute().compute()
        
        images = dask_images[0:15].compute().compute()
        print(row, column, 'success')
        break
    except:
        print(row, column, 'failed')

In [ ]:
images = dask_images[0:19].compute().compute()

In [ ]:
from macrohet.notify import send_sys_message

In [ ]:
send_sys_message()

In [ ]:
print()

In [ ]:
images.shape

In [ ]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}')

viewer.add_image(images, channel_axis = 1, 
                 # scale = napari_scale, 
                 contrast_limits=[[280, 1000],[0,3000], [0,10000]])

In [ ]:
from cellpose import models
# defining personal trained cellpose model to use
model_path = '/mnt/DATA/macrohet/upstream_development/segmentation/cellpose_training/models/macrohet_seg'
model = models.CellposeModel(gpu=True, 
                             pretrained_model=model_path)

def segment(frame, model = model, channels = [0,0], diameter = 0,#325
            min_size = 0 #2500
           ):
    
#     masks, flows, styles, diams = model.eval(frame, # for default model
#                                              channels = channels, 
#                                              diameter = diameter, 
#                                              min_size = min_size, 
#                                              )
    masks, flows, styles = model.eval(frame, # for personal model
                                      channels = channels, 
                                      diameter = diameter, 
                                      min_size = min_size, 
                                      )
    return masks


In [ ]:
frame = images[0,0,...]

In [ ]:
channels = [0,0]

masks, flows, styles = model.eval(frame, # for personal model
                                      channels = channels, 
                                      diameter = 200, 
                                      # min_size = min_size, 
                                      )

In [ ]:
viewer.add_labels(masks)

In [ ]:
import numpy as np

In [ ]:
mask_stack = []
for frame_number in tqdm(range(len(images))):
    frame = images[frame_number, 0, ...]
    masks, flows, styles = model.eval(frame, # for personal model
                                      channels = channels, 
                                      diameter = 200, 
                                      # min_size = min_size, 
                                      )
    mask_stack.append(masks)
mask_stack = np.stack(mask_stack, axis = 0)

In [ ]:
intensity_image = images[:,2]

In [ ]:
intensity_image.shape

In [ ]:
segmentation = mask_stack

In [ ]:
segmentation.shape

In [ ]:
img_props = [ 'mean_intensity',
 'max_intensity',
 'min_intensity', 'area']

In [ ]:
tracking_scale_factor = 1800/6048

In [ ]:
import btrack

In [ ]:
objects = btrack.utils.segmentation_to_objects(segmentation=segmentation,
                                               intensity_image=intensity_image, 
                                               properties=tuple(img_props),
                                               scale=(tracking_scale_factor,tracking_scale_factor),
                                               use_weighted_centroid=False, 
                                               assign_class_ID=True, 
                                             
                                           )

In [ ]:

area_threshold = 5000

filtered_objs = [obj for obj in objects if obj.properties['area'] > area_threshold]
# initialise a tracker session using a context manager

In [ ]:
 segmentation.shape[-2]*tracking_scale_factor

In [ ]:
viewer.add_labels(mask_stack)

In [ ]:
import json

In [ ]:
new_config_fn = config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'

# Load the JSON data from the original file
with open(config_fn, 'r') as json_file:
    data = json.load(json_file)

# Edit the parameters
data['TrackerConfig']['MotionModel']['prob_not_assign'] = 1  # Change the value of prob_not_assign
data['TrackerConfig']['HypothesisModel']['lambda_time'] = 6.0  # Change the value of lambda_time
data['TrackerConfig']['HypothesisModel']['lambda_dist'] = 4 #4.0   # Change the value of lambda_dist
data['TrackerConfig']['HypothesisModel']['lambda_link'] = 11 #11.0  # Change the value of lambda_link
data['TrackerConfig']['HypothesisModel']['lambda_branch'] = 0 # dont matter unless you have classifications # 55.0  # Change the value of lambda_branch
data['TrackerConfig']['HypothesisModel']['eta'] = 1e-11  # Change the value of eta
data['TrackerConfig']['HypothesisModel']['theta_dist'] = 25.0 # 25.0 # Change the value of theta_dist
data['TrackerConfig']['HypothesisModel']['theta_time'] = 6.0  # Change the value of theta_time
data['TrackerConfig']['HypothesisModel']['dist_thresh'] = 20 #45  # Change the value of dist_thresh
data['TrackerConfig']['HypothesisModel']['time_thresh'] = 3  # Change the value of time_thresh
data['TrackerConfig']['HypothesisModel']['apop_thresh'] = 0 # doesnt matter unless oyu have classificiation # 6  # Change the value of apop_thresh
data['TrackerConfig']['HypothesisModel']['segmentation_miss_rate'] = 0.05  # Change the value of segmentation_miss_rate
data['TrackerConfig']['HypothesisModel']['apoptosis_rate'] = 0 # 0.00002 # doesnt matter unless you have apop classification # Change the value of apoptosis_rate

new_config_fn = 'test_config.json' #'/home/dayn/analysis/models/btrack/particle_config_pnassign.json'
# Save the modified data to a new JSON file
with open('test_config.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)


# search radius
search_radius = 10

with btrack.BayesianTracker() as tracker:
    # configure the tracker using a config file
    tracker.configure('/home/dayn/analysis/models/btrack/particle_config_pnassign.json')
    # set max search radius
    tracker.max_search_radius = search_radius
    # define tracking method
    tracker.tracking_updates = ["MOTION", "VISUAL"]
    # redefine features so that both channels are included in track measurements
    tracker.features = ['area'] # img_props #list(filtered_objs[0].properties.keys())
    # append the objects to be tracked
    tracker.append(filtered_objs)
    # set the tracking volume
    tracker.volume=((0, segmentation.shape[-2]*tracking_scale_factor), (0, segmentation.shape[-1]*tracking_scale_factor))
    # track them (in interactive mode)
    tracker.track(step_size=25)
    # generate hypotheses and run the global optimizer
    tracker.optimize()
    # store the tracks
    tracks = tracker.tracks
# time_stamp = datetime.datetime.now().strftime("%H:%M")
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/test_nd1.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
#             writer.write_objects(objects)
        writer.write_tracks(tracks)
        # writer.write_segmentation(recolored_segmentation)
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
viewer.add_tracks(napari_tracks, name = f'tracks from', scale = (napari_scale[0]/tracking_scale_factor,napari_scale[0]/tracking_scale_factor))

In [ ]:
napari_scale = (1, 1)

In [ ]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
viewer.add_tracks(napari_tracks, name = f'tracks from nd1', scale = (napari_scale[0]/tracking_scale_factor,napari_scale[0]/tracking_scale_factor))

In [ ]:

area_threshold = 5000

filtered_objs = [obj for obj in objects if obj.properties['area'] > area_threshold]
# initialise a tracker session using a context manager

In [ ]:
# search radius
search_radius = 20

with btrack.BayesianTracker() as tracker:
    # configure the tracker using a config file
    tracker.configure('/home/dayn/analysis/models/btrack/particle_config_pnassign.json')
    # set max search radius
    tracker.max_search_radius = search_radius
    # define tracking method
    tracker.tracking_updates = ["MOTION", "VISUAL"]
    # redefine features so that both channels are included in track measurements
    tracker.features = ['area'] # img_props #list(filtered_objs[0].properties.keys())
    # append the objects to be tracked
    tracker.append(objects)
    # set the tracking volume
    tracker.volume=((0, segmentation.shape[-2]*tracking_scale_factor), (0, segmentation.shape[-1]*tracking_scale_factor))
    # track them (in interactive mode)
    tracker.track(step_size=25)
    # generate hypotheses and run the global optimizer
    tracker.optimize()
    # store the tracks
    tracks = tracker.tracks
# time_stamp = datetime.datetime.now().strftime("%H:%M")
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/test_nd1_filtered5k.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
#             writer.write_objects(objects)
        writer.write_tracks(tracks)
        # writer.write_segmentation(recolored_segmentation)
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)
viewer.add_tracks(napari_tracks, name = f'tracks from {time_stamp}', scale = (napari_scale[0]/tracking_scale_factor,napari_scale[0]/tracking_scale_factor))